In [1]:
import os, sys 
import pandas as pd
import numpy as np
os.chdir('../src')
import config
import glob
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import random
from sklearn.metrics import accuracy_score
import joblib
from sklearn.cluster import KMeans
from scipy.spatial.distance import cosine
import IPython.display as ipd

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


nn_folder = 'nn_6'

le = joblib.load(config.OUTPUT_FOLDER + '%s/label_encoder.dump'%nn_folder)

gt_train = pd.read_csv(config.DATA_FOLDER + 'gt_train.csv')

sub = {}
for flag in ['train', 'test']:
    _ = pd.read_csv(config.OUTPUT_FOLDER + '%s/pred_%s.csv'%(nn_folder, flag))
    _.rename(columns = {str(i) : le.inverse_transform(i) for i in np.arange(31)}, inplace = True)
    _['target'] = _.drop('id', 1).idxmax(axis =1)
    _.rename(columns = {'target' : 'pred_target'}, inplace = True)
    sub[flag] = _

df = gt_train[['id', 'target']].merge(sub['train'], on = 'id')
assert len(df) == len(gt_train)

accuracy_score(df['target'], df['pred_target'])

In [319]:
nn_folder = 'stacker_xgb_7'
sub = {}
for flag in ['train', 'test']:
    _ = pd.read_csv(config.OUTPUT_FOLDER + '%s/sub_%s.csv'%(nn_folder, flag))
    _.rename(columns = {'target' : 'pred_target'}, inplace = True)
    _['confidence'] = _[config.mapping_dict.keys()].max(axis = 1)
    gt = pd.read_csv(config.DATA_FOLDER + 'gt_%s_stats.csv'%flag)
    _ = _.merge(gt, on = 'id')
    sub[flag] = _
    assert len(_) == len(gt)


In [160]:
feature_columns = config.mapping_dict.keys()
# feature_columns.remove('sheila')

centroid_centers = {}
for c in config.mapping_dict.keys():#config.allowed_train_labels: #config.mapping_dict.keys():
    centroid_centers[c] = df[df['target'] == c][feature_columns].median().values
    
centroid_centers['unknown'] = df[~df['target'].isin(config.allowed_train_labels)][feature_columns].median().values    

In [321]:
sub['train'].head()

,id,pred_target,bed,bird,cat,dog,down,eight,five,four,...,yes,zero,confidence,path,person_id,target,stat_max_audio,stat_min_audio,stat_mean_audio,stat_std_audio
0,73af0c50_nohash_1_wow,wow,0.016028,0.016050,0.016027,0.016048,0.016048,0.016029,0.016025,0.016044,...,0.016012,0.016020,0.519035,/home/tony/ssd_buffer/kaggle_speech/train/audi...,73af0c50,wow,2768,-3298,-0.015562,504.531799
1,187af8be_nohash_1_wow,wow,0.016028,0.016050,0.016027,0.016048,0.016048,0.016029,0.016025,0.016044,...,0.016012,0.016020,0.519035,/home/tony/ssd_buffer/kaggle_speech/train/audi...,187af8be,wow,4036,-4305,-0.355500,659.334380
2,c4533c47_nohash_1_wow,wow,0.016028,0.016050,0.016027,0.016048,0.016048,0.016029,0.016025,0.016044,...,0.016012,0.016020,0.519035,/home/tony/ssd_buffer/kaggle_speech/train/audi...,c4533c47,wow,10094,-6106,-0.731265,1241.821693
3,e57abea3_nohash_1_wow,wow,0.015892,0.015913,0.015891,0.015912,0.015912,0.015893,0.015889,0.015907,...,0.015876,0.015884,0.514628,/home/tony/ssd_buffer/kaggle_speech/train/audi...,e57abea3,wow,7797,-8634,35.931875,1571.973819
4,baeac2ba_nohash_0_wow,wow,0.016028,0.016050,0.016027,0.016048,0.016048,0.016029,0.016025,0.016044,...,0.016012,0.016020,0.519035,/home/tony/ssd_buffer/kaggle_speech/train/audi...,baeac2ba,wow,6611,-7420,4.955375,802.042799


In [325]:
N = 467
def get_item(df_):
    items = df_[(df_['confidence'] <= 0.055) & (df_['confidence'] <= 0.055)]
#  df_[(df_['pred_target'].isin(config.allowed_train_labels))]
    audio_id = items['id'].values[N]
    confidence = items['confidence'].values[N]
    volume = items['stat_max_audio'].values[N]
    print('Predicted', items['pred_target'].values[N], confidence, volume)
    item = items[feature_columns].values[N]
    return item, audio_id
item, audio_id = get_item(sub['test'])
ipd.Audio(config.DATA_FOLDER + 'test/audio/%s.wav'%audio_id) # load a local WAV file

('Predicted', 'nine', 0.047866329550699999, 3)


In [350]:
low_volume_ids = sub['test'][(sub['test']['stat_max_audio'] <= 200) & (sub['test']['confidence'] <= 0.055)]['id'].tolist()

In [351]:
submission = sub['test'][['id', 'pred_target']].copy()

In [352]:
submission.loc[submission['id'].isin(low_volume_ids), 'pred_target'] = 'silence'
submission['label'] = submission['pred_target']
submission['label'] = submission['label'].apply(lambda x: x if x in config.allowed_train_labels else 'unknown')
submission['fname'] = submission['id'] + '.wav'

In [353]:
submission['label'].value_counts()

unknown    84840
silence    14635
up          6608
left        6416
no          6213
on          5966
off         5952
go          5698
down        5619
right       5563
yes         5526
stop        5502
Name: label, dtype: int64

In [349]:
submission[['fname', 'label']].to_csv('stacker_xgb_7_volume200_as_silence_condifence0055.csv', index = False)

In [242]:
sub['test']['confidence'].quantile(0.03)

0.06234527844933901

In [179]:
num_iter = 25
inds = np.random.randint(low=0, high=len(item)-1, size=(num_iter,12))

In [326]:
max_distance = 0
max_label = 'nan'

ans = []
for i in np.arange(num_iter):
    for c in centroid_centers.keys():
        a = item[inds[i,:]]
        b = centroid_centers[c][inds[i,:]]
        distance = 1 - cosine(a, b)
#         distance = eculid(a, b)
        if distance > max_distance: 
            max_label = c
            max_distance = distance
    ans.append({'label' : max_label, 'distance' : max_distance})
ans = pd.DataFrame(ans).groupby('label').agg({'distance' :['count', 'mean']}).reset_index()
ans.columns = ['_'.join(col) for col in ans.columns]
ans['distance_count'] = ans['distance_count']/(ans['distance_count'].sum())
ans.sort_values('distance_count', ascending= False)

#     print(' max', max_label, max_distance)

,label_,distance_count,distance_mean
4,tree,0.60,0.988764
3,three,0.20,0.974484
1,on,0.08,0.978136
2,silence,0.08,0.981029
0,no,0.04,0.976678


In [309]:
silence

[]

In [ ]:
sorted(config.allowed_train_labels)

In [ ]:
random.sample(feature_columns,4)

In [ ]:
feature_columns

In [ ]:
config.allowed_train_labels